<h1> Merging of Invoice Header and Invoice Lines

In [58]:
import pandas as pd

<h2> Importing Invoice Header and Invoice Lines CSVs


In [59]:
#Import inv_hd dataframe from inv_hd.csv
inv_hd = pd.read_csv('inv_hd.csv')
inv_hd.head()

,store_id,invoice_id,supplier_number,invoice_number,return_invoice,status,method,cod,user_id,external_po_number,...,flags,base_total_qty,base_total_net,final_total_qty,final_misc_charge,final_misc_allowance,final_total_net,final_total_retail,num_line_items,i_audit
0,410087,170,1771394,121590806841440,False,2,2,False,76013.0,NULL,...,12289,100.0,100.00,100.0,0.0,0.0,1170.00,19900.00,1,False
1,410085,256,9179773,797761846933028,False,2,5,False,NaN,NULL,...,12289,4.0,14.69,4.0,0.0,0.0,14.84,21.96,2,False
2,410088,257,9179773,794268529783944,False,2,5,False,NaN,NULL,...,12289,158.0,718.12,158.0,0.0,0.0,725.37,9789.68,40,False
3,340028,275,4977635,974786101227895,False,2,5,False,23098.0,NULL,...,12288,1.0,119.95,12.0,0.0,0.0,119.95,155.88,1,False
4,410055,276,4977635,459915081072217,False,2,5,False,66734.0,NULL,...,12288,1.0,-295.90,198.0,0.0,0.0,3833.05,10212.60,30,False


In [60]:
#Import inv_line dataframe from inv_line.csv
inv_line = pd.read_csv('inv_line.csv')
inv_line.head()

,store_id,invoice_id,line_number,container_id,sku_number,exception,flags,base_qty,final_qty,base_um,...,base_cost,final_cost,base_allowance,final_allowance,billing_price,billing_price_mult,billing_major_dept,billing_dept_number,store_cost,store_allowance
0,340005,170,1,NaN,9881155442438,0,257.0,100.0,100.0,C,...,11.70,11.70,0.0,0.0,19.90,1.0,101.0,5.0,11.70,0.0
1,340024,256,1,NaN,7904555584739,0,36.0,2.0,2.0,E,...,3.72,3.72,0.0,0.0,5.99,1.0,1056.0,56.0,3.72,0.0
2,340024,256,2,NaN,4653609369867,256,36.0,2.0,2.0,E,...,3.70,3.70,0.0,0.0,4.99,1.0,1056.0,56.0,3.95,0.0
3,340007,257,1,NaN,6428855040557,0,36.0,1.0,1.0,E,...,3.85,3.85,0.0,0.0,3.99,1.0,719.0,57.0,3.85,0.0
4,340007,257,2,NaN,3523821409384,0,36.0,10.0,10.0,E,...,3.70,3.70,0.0,0.0,3.99,1.0,719.0,19.0,3.70,0.0


<h2> Further cleaning of dataframes for merging

<h3> These inv_line columns (new names) contain detailed information for the matching columns within inv_hd

In [61]:
#Creation of a list of the old inv_line column names and the new inv_line column names.
line_col_names = {
    'line_number': 'num_line_items',
     'base_qty': 'base_total_qty',
     'final_qty': 'final_total_qty',
     'base_cost': 'base_total_net',
     'final_cost': 'final_total_net',
     'final_allowance': 'final_misc_allowance',
     'billing_price': 'final_total_retail',
     'billing_major_dept': 'major_dept_number'
}

In [74]:
#Applying the new inv_line column names
inv_line = inv_line.rename(columns=line_col_names)

In [75]:
#Printing column names from both inv_hd and inv_line to make sure name changes took place and match
print("inv_hd columns:", inv_hd.columns.tolist())
print("inv_line columns:", inv_line.columns.tolist())

inv_hd columns: ['store_id', 'invoice_id', 'supplier_number', 'invoice_number', 'return_invoice', 'status', 'method', 'cod', 'user_id', 'external_po_number', 'major_dept_number', 'invoice_date', 'delivery_timestamp', 'check_in_timestamp', 'final_timestamp', 'flags', 'base_total_qty', 'base_total_net', 'final_total_qty', 'final_misc_charge', 'final_misc_allowance', 'final_total_net', 'final_total_retail', 'num_line_items', 'i_audit']
inv_line columns: ['store_id', 'invoice_id', 'num_line_items', 'container_id', 'sku_number', 'exception', 'flags', 'base_total_qty', 'final_total_qty', 'base_um', 'final_um', 'base_pack_qty', 'final_pack_qty', 'base_total_net', 'final_total_net', 'base_allowance', 'final_misc_allowance', 'final_total_retail', 'billing_price_mult', 'major_dept_number', 'billing_dept_number', 'store_cost', 'store_allowance']


<h3> Many of the inv_line rows are missing the major_dept_number for the overall invoice. For reporting reasons, it's better to get the major dept for as many of these as possible. Invoice types like Total Only will not have invoice lines for us to do this with. 

In [73]:
# Group the inv_line by 'invoice_id' and calculate the maximum 'billing_dept_number'
max_dept_number = inv_line.groupby(['store_id','invoice_id'])['billing_dept_number'].transform('max')

# Assign the maximum 'billing_dept_number' to the 'major_dept_number' column
inv_line['major_dept_number'] = max_dept_number

In [64]:
# Creating a filtered_df for store 340007 and invoice_id 257 to determine accuracy
filtered_df = inv_line[(inv_line['store_id'] == 340007) & (inv_line['invoice_id'] == 257)]

# Selecting the columns necessary from filtered_df for verification
columns_to_print = ['store_id', 'invoice_id', 'billing_dept_number', 'major_dept_number']

# Print the above columns of filtered_df for review
print(filtered_df[columns_to_print])

    store_id  invoice_id  billing_dept_number  major_dept_number
3     340007         257                 57.0               57.0
4     340007         257                 19.0               57.0
5     340007         257                 57.0               57.0
6     340007         257                 57.0               57.0
7     340007         257                 57.0               57.0
8     340007         257                 57.0               57.0
9     340007         257                 57.0               57.0
10    340007         257                 57.0               57.0
11    340007         257                 19.0               57.0
12    340007         257                 57.0               57.0
13    340007         257                 57.0               57.0
14    340007         257                 57.0               57.0
15    340007         257                 57.0               57.0
16    340007         257                 57.0               57.0
17    340007         257 

In [65]:
#Further verification of this working by getting the count of each dept_number for filtered_df
count_19 = (filtered_df['billing_dept_number'] == 19).sum()
count_57 = (filtered_df['billing_dept_number'] == 57).sum()
print(count_19, count_57)

19 21


<h2> Validating inv_hd and inv_line data types match for merging

In [66]:
#Printing inv_hd columns data types to compare against inv_line
inv_hd_types = inv_hd.dtypes
print(inv_hd_types)

store_id                  int64
invoice_id                int64
supplier_number           int64
invoice_number            int64
return_invoice             bool
status                    int64
method                    int64
cod                        bool
user_id                 float64
external_po_number       object
major_dept_number       float64
invoice_date             object
delivery_timestamp       object
check_in_timestamp       object
final_timestamp          object
flags                     int64
base_total_qty          float64
base_total_net          float64
final_total_qty         float64
final_misc_charge       float64
final_misc_allowance    float64
final_total_net         float64
final_total_retail      float64
num_line_items            int64
i_audit                    bool
dtype: object


In [67]:
#Printing inv_line columns data types to compare against inv_hd
inv_line_types = inv_line.dtypes
print(inv_line_types)

store_id                  int64
invoice_id                int64
num_line_items            int64
container_id            float64
sku_number                int64
exception                 int64
flags                   float64
base_total_qty          float64
final_total_qty         float64
base_um                  object
final_um                 object
base_pack_qty           float64
final_pack_qty          float64
base_total_net          float64
final_total_net         float64
base_allowance          float64
final_misc_allowance    float64
final_total_retail      float64
billing_price_mult      float64
major_dept_number       float64
billing_dept_number     float64
store_cost              float64
store_allowance         float64
dtype: object


<h3> inv_line and inv_hd flags column's data types do not match, correcting this. 

In [68]:
#Updating inv_line flags column to int64
inv_line['flags'] = inv_line['flags'].astype('int64')

In [69]:
#Reprint of the inv_line column data types to make sure change above took place
inv_line_types = inv_line.dtypes
print(inv_line_types)

store_id                  int64
invoice_id                int64
num_line_items            int64
container_id            float64
sku_number                int64
exception                 int64
flags                     int64
base_total_qty          float64
final_total_qty         float64
base_um                  object
final_um                 object
base_pack_qty           float64
final_pack_qty          float64
base_total_net          float64
final_total_net         float64
base_allowance          float64
final_misc_allowance    float64
final_total_retail      float64
billing_price_mult      float64
major_dept_number       float64
billing_dept_number     float64
store_cost              float64
store_allowance         float64
dtype: object


<h2> Merging of inv_hd and inv_line

In [72]:
# Get the common columns between inv_hd and inv_line
common_columns = list(set(inv_hd.columns).intersection(inv_line.columns))

# Create a inv_sum with all columns from inv_hd
inv_sum = inv_hd.copy()

# Select only the common columns from inv_line
inv_line_common_columns = inv_line[common_columns]

# Concantenate the rows from inv_line to inv_sum
inv_sum = pd.concat([inv_sum, inv_line_common_columns], ignore_index=True)

# Sort the inv_sum by 'store_id' and 'invoice_id'
inv_sum = inv_sum.sort_values(by=['store_id', 'invoice_id'])

# Print inv_sum
print(inv_sum)

       store_id  invoice_id  supplier_number  invoice_number return_invoice  \
1275     340001         301        9068305.0    4.191908e+12          False   
775      340001         305        3629745.0    2.262191e+14          False   
1707     340001         305              NaN             NaN            NaN   
1708     340001         305              NaN             NaN            NaN   
275      340001         309        5963353.0    8.402456e+14          False   
...         ...         ...              ...             ...            ...   
33166    410100         628              NaN             NaN            NaN   
33167    410100         628              NaN             NaN            NaN   
33168    410100         628              NaN             NaN            NaN   
33169    410100         628              NaN             NaN            NaN   
1005     410100         714        9179773.0    9.086860e+12          False   

       status  method    cod  user_id      external